In [1]:
import os
import uuid
import random
import ast
import base64
import json

import boto3
import pandas as pd

REGION = boto3.session.Session().region_name

## Load the data from local storage

In [2]:
df = pd.read_csv("Fake and Real News Dataset/True.csv")

In [3]:
def get_record(df):
    """
    This is a generator to get a random record from the source dataframe. Also, converts
    the data into bytes object. Data Streams accepts only bytes datatypes.
    """
    while True:
        record = df.sample(1).to_dict()
        record = {
            "text": "".join(record['text'].values()),
        }
        record = str(record).encode('utf-8')
        yield record

record_generator = get_record(df) # Initialize the generator

### Descriptive stats of the Real news data

In [81]:
df.text.apply(lambda x:len(x.split())).describe()

count    21417.000000
mean       385.640099
std        274.006204
min          0.000000
25%        148.000000
50%        359.000000
75%        525.000000
max       5172.000000
Name: text, dtype: float64

## PUT data into Kinesis Streams (randomly)

In [105]:
PARTITION_KEYS = [
    'e3d3c5a376e74bbc93ce1ca675f1a021', 
    'def4af889a8d427cb59f7aad7fb6bd24', 
    '1af95ab90bdf4c3490f7ab52ca4c0b65', 
    'd1cf18d92f544c0oo18f30a0a0378e3b',
    'd1cf18d92f544c0681uu30a0a0378e3b',
    'd1cf18dttf544c06818f30a0a0378e3b',
    'd1cf18dtyf544c06818f30a0a0378e3b',
    'd1cf18d9df544c06818f3hh0a0378e3b',
    'd1cfred92f544c06818f30a0a0378e3b',
    'drrf18d92f544c06818f30a0a0378e3b',
    'd1c568d92f544c06818f30a0a0378e3b',
] # Streams putObject API requires users to specify a Partition parameter

STREAM_NAME = "news-stream" # Name of the Kinesis Stream


kinesis_client = boto3.client('kinesis') # Kinesis client

In [107]:
for _ in range(10):    
    put_params = {
        'StreamName': STREAM_NAME,
        'Data': next(record_generator),
        'PartitionKey': random.choice(PARTITION_KEYS)
    }

    try: 
        assert STREAM_NAME in kinesis_client.list_streams()['StreamNames']
        print(put_params)
        kinesis_client.put_record(**put_params)
    except AssertionError as e:
        print(f"Assertion Error: Stream '{STREAM_NAME}' is not found in region {REGION}")

{'StreamName': 'news-stream', 'Data': b"{'text': 'ISTANBUL/KHARTOUM (Reuters) - Turkey will rebuild a ruined Ottoman port city on Sudan s Red Sea coast and construct a naval dock to maintain civilian and military vessels, Sudan s foreign minister said on Tuesday, as Ankara expands military and economic ties in Africa. The restoration at Suakin was agreed during a visit to the ancient port by Turkish President Tayyip Erdogan, Foreign Minister Ibrahim Ghandour said. Making the first trip by a Turkish president to Sudan, Erdogan said Turkey had been temporarily granted part of Suakin so it could rebuild the area as a tourist site and a transit point for pilgrims crossing the Red Sea to Mecca. He said the Suakin deal was one of several, worth $650 million in total, agreed with Sudan, which emerged from two decades of U.S. sanctions in October and is seeking to attract international investment. The countries also agreed  to build a dock to maintain civilian and military vessels,  Ghandour t

{'StreamName': 'news-stream', 'Data': b"{'text': 'WASHINGTON (Reuters) - U.S. President Donald Trump will meet April 6-7 with Chinese President Xi Jinping at the president\xe2\x80\x99s Mar-a-Lago retreat in Florida, a source familiar with the meeting said on Tuesday. It will be their first face-to-face meeting since Trump took office on Jan. 20. Trump played host to Japanese Prime Minister Shinzo Abe last month at Mar-a-Lago, in Palm Beach, Florida. '}", 'PartitionKey': 'd1cf18d92f544c0681uu30a0a0378e3b'}
{'StreamName': 'news-stream', 'Data': b"{'text': 'LONDON (Reuters) - An aardvark was killed and four meerkats are missing believed dead after a blaze tore through part of London Zoo early on Saturday, destroying a cafe and gift shop. The fire broke out shortly after 0600 GMT at the zoo s Animal Adventure section before spreading to the shop and cafe, near to an area where visitors can handle and feed animals, the zoo and London Fire Brigade said. More than 70 firefighters took three h

{'StreamName': 'news-stream', 'Data': b"{'text': 'BERLIN (Reuters) - Germany\xe2\x80\x99s Economy Minister Sigmar Gabriel said on Sunday that talks on the Transatlantic Trade and Investment Partnership (TTIP), a free trade deal being negotiated by the United States and the European Union, had essentially failed. \xe2\x80\x9cThe negotiations with the USA have de facto failed because we Europeans did not want to subject ourselves to American demands,\xe2\x80\x9d he said, according to a written transcript from German broadcaster ZDF of an interview due to be broadcast on Sunday. \xe2\x80\x9cThings are not moving on that front,\xe2\x80\x9d said Gabriel, who is also Germany\xe2\x80\x99s vice chancellor. The U.S. and the EU have been negotiating the TTIP for three years and both sides had sought to conclude talks in 2016 but they have differences over various issues, including agriculture. '}", 'PartitionKey': 'drrf18d92f544c06818f30a0a0378e3b'}
{'StreamName': 'news-stream', 'Data': b"{'text

## Sample Data

In [60]:
records_list = [{'text': 'WASHINGTON (Reuters) - A U.S. judge on Thursday warned lawyers for Paul Manafort and Richard Gates, former advisers to President Donald Trump’s presidential campaign, not to speak about their case outside of court, saying it could interfere with the ability to conduct a fair trial. “This is a criminal trial and not a public relations campaign,” Judge Amy Berman Jackson said during a hearing on the case. The judge said initial bail terms for both men would remain in place for now, and she set a bail hearing for Monday to consider changes. She also indicated an openness to a proposal from the defense to begin the trial in April, and said it was possible a tentative trial date could be set on Monday. ', 'timestamp': '2023-07-27 04:38:39.122000', 'id': '6a7abdad50b84bb19cb237c2d386e786'}, {'text': 'NEW YORK (Reuters) - New Jersey Governor Chris Christie took a drubbing on social media for his demeanor in an appearance with Republican presidential front-runner Donald Trump at a televised victory speech and news conference on Super Tuesday. Adding to his woes, six New Jersey newspapers called on Christie to resign, partly for endorsing Trump after dropping out of the race to become the Republican Party’s candidate in November’s election. A Christie spokesman, Brian Murray, declined to comment immediately on the editorial, which appeared in newspapers owned by Gannett Co Inc (GCI.N). Christie surprisingly endorsed Trump on Friday, calling the billionaire New York businessman “the best person to beat Hillary Clinton in November.” Some questioned whether Christie’s enthusiasm was genuine following his appearance at Trump’s event in Palm Beach, Florida on Tuesday night. Trump appeared before TV cameras after news media organizations declared him the winner of some of the state-by-state primaries and caucuses and as Super Tuesday results continued rolling in. Christie introduced Trump, then stood behind and just to the right of the brash billionaire. The governor stared blankly ahead, at times looking troubled, as Trump took reporters’ questions. The reaction to Christie’s demeanor was swift and unforgiving on social media. “This is what a man collapsing under the weight of his own shame looks like,” Sean D. Illing (@sean_illing) tweeted on Tuesday. “Did #ChrisChristie look pathetic standing behind #Trump, or what?!?”  Karen Walz (@khwalz) tweeted on Wednesday. News website BuzzFeed on Tuesday evening tweeted a clip of the video footage set to the classic Simon and Garfunkel song “Sounds of Silence,” with the caption “Hello darkness, my old friend,” a lyric from the song. Christie spokesman Murray would not comment on the social media reaction, saying in an email, “I’ll let you know when we opt to slide down into that muck.”    In the editorial calling on Christie to resign, the six newspapers, including the Asbury Park Press, called his endorsement hypocritical. “We’re disgusted with (Christie’s) endorsement of Donald Trump after he spent months on the campaign trail trashing him, calling him unqualified by temperament and experience to be president,” the editorial read. Christie, serving the second of two four-year terms, is not slated to leave office until December 2017.  (Reporting By Amy Tennery in New York; Additional reporting by Gina Cherelus in New York) This article was funded in part by SAP. It was independently created by the Reuters editorial staff. SAP had no editorial involvement in its creation or production.', 'timestamp': '2023-07-27 04:38:39.246000', 'id': 'd889253248614d4d9783d0cbb9b85e3e'}, {'text': 'PHILADELPHIA (Reuters) - When President Donald Trump was elected last November, Republican lawmakers enthusiastically joined his call to rewrite the tax code and dismantle Obamacare in the first 100 days of his presidency. But as congressional Republicans gathered for an annual policy retreat in Philadelphia on Wednesday, the 100-day goal morphed into 200 days. As the week wore on, leaders were saying it could take until the end of 2017 - or possibly longer - for passage of final legislation. Trump had a different idea when he spoke to lawmakers in Philadelphia, telling them: Enough talk. Time to deliver. The divergent views on the timetable were among many indications of tensions that simmered just below the surface at the three-day Republican retreat. Before the cameras, Trump and Republicans sought to convey an image of a happy, unified family, playing down differences over tax policy, whether to reinstate torture interrogation techniques and investigating 2016 election fraud. And clearly there is none of the open warfare that has sometimes erupted among Republicans, such as when Senator Ted Cruz infuriated many of his colleagues by leading a standoff over Obamacare that partially shut down the government in 2013. But barely visible in Philadelphia, there are potential flashpoints of disagreement within the Republican rank-and-file in Congress as well as between Republican lawmakers and the unorthodox new president. These include how and when to replace Obamacare if Republicans succeed in their quest to repeal it; how to revamp the multi-layered tax code, whether to build a wall on the U.S. border with Mexico and the nature of the U.S. relationship with Russia. When it comes to tax reform, senior congressional aides said the spring of 2018 might be a more likely time than this year for the passage of legislation. Republican lawmakers lavished praise on Trump in public. In dozens of interviews, many said they felt he would be an energetic champion of issues they cared about. But some also voiced fears that his big agenda would drive up deficits and said they were still searching for details on his plans. Several Republican lawmakers and aides said they were wary of the cost of his plan to build a wall on the border with Mexico. Republican leaders have said the wall proposal under discussion would cost $12 billion to $15 billion cost but some congressional aides say it could end up easily topping $20 billion. Republican Representative Will Hurd, whose Texas district partially borders Mexico, went a step further, calling the wall an ineffective tool for stopping illegal immigration. Others warned a border adjustment tax on foreign goods to pay for that wall could hurt U.S. companies’ profits, raise costs for American consumers and spark retaliation by foreign trading partners. Some lawmakers also worry that some of their constituents could be at risk of losing healthcare coverage if the push to repeal Obamacare moves too quickly. Republican Representative Tom Cole said rank-and-file lawmakers have an incentive to fall in line behind Trump. “You don’t want to be the reason why we weren’t successful in getting these things done,” he said. Still, Cole said Republicans are taking stock of the potential cost of the biggest items on Trump’s agenda such as the wall, infrastructure projects, tax cuts and beefing up military spending. “I think they worry about it,” Cole said.  Following Trump’s speech to the lawmakers on Thursday, Senator James Risch said that no decisions had been made on the replacement of Obamacare, a complex law that has expanded healthcare insurance to millions of Americans. “It’s going to take a while to resolve it,” Risch said. Asked by reporters whether Republicans had a clear idea of what Trump would like to replace Obamacare with, Risch responded, “In detail, no.” ', 'timestamp': '2023-07-27 04:38:39.409000', 'id': '7c85c419af68462b86404eaca912e607'}, {'text': 'WASHINGTON (Reuters) - Republican presidential hopeful Ted Cruz lashed out at rival candidate Donald Trump on the day of Indiana’s primary on Tuesday, calling him a pathological liar, narcissist and bully after the front-runner linked Cruz’s father to John F. Kennedy’s assassin. “I’m going to tell you what I really think of Donald Trump. This man is a pathological liar. He doesn’t know the difference between truth and lies,” Cruz told reporters in a scathing indictment. “He lies practically every word that comes out of his mouth. And...his response is to accuse everybody else of lying.” In a television appearance earlier, Trump linked Cruz’s father, Rafael Cruz, to Kennedy assassin Lee Harvey Oswald, citing a report by National Enquirer, a supermarket tabloid known for celebrity gossip and alien sightings. “You know, his father was with Lee Harvey Oswald prior to Oswald’s being — you know, shot. I mean, the whole thing is ridiculous. What is this, right prior to his being shot, and nobody brings it up,” Trump said on “Fox & Friends.”  Republicans in Indiana might give New York billionaire Trump an almost unstoppable advantage in his turbulent journey toward the party’s presidential nomination.  The real estate mogul holds a double-digit polling lead in the Midwestern state over Cruz, a U.S. Senator of Texas.    Cruz denounced the Enquirer and its chief executive, David Pecker, who has endorsed Trump, for claiming his father, a Cuban immigrant to the United States, was pictured handing out pro-Fidel Castro literature with Oswald in New Orleans in 1963. “Yes, my dad killed JFK, he is secretly Elvis and Jimmy Hoffa is buried in his back yard,” Cruz said in a sarcastic retort. Cruz also unleashed fury over Trump’s earlier comment about his wife, Heidi. “Donald has a real problem with women. Donald is terrified by strong women - he lashes out at them,” Cruz said.  “The man is utterly amoral. Morality does not exist for him,” Cruz said. ', 'timestamp': '2023-07-27 04:38:39.598000', 'id': '8a64b013e1024784b2b360a14a0ef70e'}, {'text': 'HARARE (Reuters) - Former Zimbabwe finance minister Tendai Biti said on Thursday he would be happy to work in any national unity government that emerges from this week s coup, but only if opposition leader Morgan Tsvangirai was also on board.  If Morgan says he s in, I m in,  Biti, who earned international respect during his time as finance minister in a 2009-2013 unity government, told Reuters.  The country needs a solid pair of hands so one might not have a choice.  ', 'timestamp': '2023-07-27 04:38:41.617000', 'id': 'c2bcb78b18ca433d9cb95d062fa79f15'}]

<module 'lambdas.receive_stream' from 'C:\\Users\\gskps\\OneDrive\\Documents\\Repositories\\comprehend-kinesis-stream\\lambdas\\receive_stream.py'>